<a href="https://colab.research.google.com/github/ArnavDhiman/NLP_dataset_QA_expression/blob/main/NLP_dataset_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# This code works on v7w pointing dataset json and converts the Questions to refering expressions
# by using pos tagging and removing some tags to form a refering expressions. 
# Manually checking the expressions generated this method gives an accuracy of near to 75%
# and fails on the questions that are complex or have more than 3 nouns in them.

# Install stanza; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

# Import stanza
import stanza

import json

     |████████████████████████████████| 235kB 8.6MB/s 


In [3]:
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

2020-10-25 00:23:47 INFO: Installing CoreNLP package into ./corenlp...
2020-10-25 00:43:12 WARNING: For customized installation location, please set the `CORENLP_HOME` environment variable to the location of the installation. In Unix, this is done with `export CORENLP_HOME=./corenlp`.


In [4]:
# Examine the CoreNLP installation folder to make sure the installation is successful
!ls $CORENLP_HOME

build.xml				  jollyday.jar
corenlp.sh				  LIBRARY-LICENSES
CoreNLP-to-HTML.xsl			  LICENSE.txt
ejml-core-0.38.jar			  Makefile
ejml-core-0.38-sources.jar		  patterns
ejml-ddense-0.38.jar			  pom-java-11.xml
ejml-ddense-0.38-sources.jar		  pom.xml
ejml-simple-0.38.jar			  protobuf.jar
ejml-simple-0.38-sources.jar		  README.txt
input.txt				  RESOURCE-LICENSES
input.txt.out				  SemgrexDemo.java
input.txt.xml				  ShiftReduceDemo.java
javax.activation-api-1.2.0.jar		  slf4j-api.jar
javax.activation-api-1.2.0-sources.jar	  slf4j-simple.jar
javax.json-api-1.0-sources.jar		  stanford-corenlp-4.1.0.jar
javax.json.jar				  stanford-corenlp-4.1.0-javadoc.jar
jaxb-api-2.4.0-b180830.0359.jar		  stanford-corenlp-4.1.0-models.jar
jaxb-api-2.4.0-b180830.0359-sources.jar   stanford-corenlp-4.1.0-sources.jar
jaxb-core-2.3.0.1.jar			  StanfordCoreNlpDemo.java
jaxb-core-2.3.0.1-sources.jar		  StanfordDependenciesManual.pdf
jaxb-impl-2.4.0-b180830.0438.jar	  sutime
jaxb-impl-2.4.0-b180830.0438-sources

In [5]:
# Import client module
from stanza.server import CoreNLPClient

In [6]:
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(
    annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner'], 
    memory='4G', 
    endpoint='http://localhost:9001',
    be_quiet=True)
print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()
import time; time.sleep(10)

2020-10-25 00:43:12 INFO: Writing properties to tmp file: corenlp_server-52187fce0c4c47dc.props
2020-10-25 00:43:12 INFO: Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-52187fce0c4c47dc.props -annotators tokenize,ssplit,pos,lemma,ner -preload -outputFormat serialized


In [8]:
# import v7w pointing data from files
# when promted to upload a json file... upload dataset_v7w_pointing.json only
# downlad the file from http://ai.stanford.edu/~yukez/visual7w/
# downlod the pointing data
# or download the file directly by clicking this link and extract the json"from the zip
#http://ai.stanford.edu/~yukez/papers/resources/dataset_v7w_pointing.zip

from google.colab import files
uploaded = files.upload()

Saving dataset_v7w_pointing.json to dataset_v7w_pointing.json


In [9]:
# import io
file_name = "dataset_v7w_pointing.json"

In [10]:
data = json.loads(uploaded[file_name].decode("utf-8"))

In [12]:
#separate images_qa and segmantaion boxes

images = data['images']
boxes = data['boxes']


In [35]:
## Create POS tag for approx 2000 questions using stanza

# Store the bounding box ids for top 2000 questions
boundingBox_ids = {}

# variable to count 2000 qustions and break the loop after that
nn_questions = 0
imageDataset = []
for image in images:
  if nn_questions > 2000:
    break;

  # create a data structure similar to the original v7w pointing dataset
  imageJson = {}
  imageJson['filename'] = image['filename']
  imageJson['reference_pairs'] = []
  for qa in image['qa_pairs']:
    newJsonData = {}    
    res = ""
    nn_count = 0
    first_nn_index = -1

    # Annotate some text
    text =  qa['question']
    document = client.annotate(text)
    for i, sent in enumerate(document.sentence):
        for t in sent.token:
          # remove all the pos tags that are ".", "VBZ", "WDT" and "VBP"
          # "VBP"  will only be removed if the previous word was a noun
          if t.pos != "." and t.pos != "WDT" and t.pos != "VBZ":
            if t.pos == "VBP" and first_nn_index - 1 == t.beginIndex:
              continue
            res= res+ " " + t.word
          if t.pos == 'NN':
            if first_nn_index == -1:
              first_nn_index = t.beginIndex
            nn_count += 1

    # Select the sentence only if the count of nouns is 2
    # This is done by trial and error
    # 2 NN produces more number of gramatically accurate sentences

    if nn_count != 2:
      res = ""

    if res != "":
      boundingBox_ids[qa['answer']] = 1
      nn_questions+=1
      newJsonData['boundingBox_id'] = qa['answer']
      newJsonData['reference'] = res[1:]

    if len(newJsonData) > 0:
      imageJson['reference_pairs'].append(newJsonData)
  if imageJson['reference_pairs']:
    imageDataset.append(imageJson)  

2020-10-25 01:14:23 INFO: Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-52187fce0c4c47dc.props -annotators tokenize,ssplit,pos,lemma,ner -preload -outputFormat serialized


In [46]:
boxesData = []

# select the bounding boxes only for the 1000 questions that are filtered above
for b in boxes:
  if b['box_id'] in boundingBox_ids:    
    boxesData.append(b)

In [47]:
# print(boxesData)

In [48]:
#store the data in a similar data structure as the original data

newData = {
    'images' : imageDataset,
    'boxes' : boxesData
}

# print(newData)

In [49]:

# print(newData['boxes'])

In [45]:
#download the JSON file

out_file = open('changed_dataset_v7w_pointing.json', 'w')
newJsonDataFile = json.dump(newData, out_file, indent =4)
files.download('changed_dataset_v7w_pointing.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Shut down the background CoreNLP server
client.stop()

# time.sleep(10)
!ps -o pid,cmd | grep java